## EDA Project

**What sales insights can we gather from advertisement data?** We've been provided with data about used car advertisements over the course of approximately a year: information about the cars, their prices, when they were listed, and how quickly they sold.

For this project, I want to look at a small subset of the data, pickup trucks, and compare sale prices and days listed for popular makes and models.

In [622]:
import pandas as pd
import plotly.express as px

vehicles = pd.read_csv('../vehicles_us.csv',parse_dates=['date_posted'])

vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 5.1+ MB


### Cleaning the data

A preliminary `info()` call shows that we have some missing values in the following fields: `model_year`, `cylinders`, `odometer`, `paint_color`, and `is_4wd`. We can find substitutes for the missing values, but it'll help if we can clean up the data we *do* have first. I'll start by looking through each field.

It appears that besides the missing values, there is some bad data in `price`. I'm not necessarily worried about the 20-year-old Nissan Frontier purportedly going for \$375,000, but there are a little over 1000 rows where `price` is under \$500, and for almost 800 of those, `price` is 1. Many sub-\$500 cars are described as being in 'excellent' or 'like new' condition. It seems likely that some of these low values are missing a zero, or represent monthly payment amounts, not total sale price. To improve the accuracy of my price analysis, I'm going to **exclude vehicles where `price` is missing (1) or below \$500.** There will likely be other vehicles in the 500-1000 range where the `price` is actually inaccurate, but since there are also poor-condition vehicles that sell in this range, I can't determine which prices are accurate without cherry-picking. Better to have a little bad data than a lot.

In [593]:
#vehicles.query('price < 500').sort_values(by='price', ascending=False)

vehicles = vehicles[vehicles['price'] >= 500]

Listing values in the `model_year` field, most appeared to be valid, although there were a few outliers: very old model years. Most rows with a `model_year` prior to 1970 are plausible classic cars. There are just a handful of obvious errors at the bottom.

In [594]:
print(vehicles[vehicles['model_year'] < 1950].sort_values(by='model_year').to_string())

       price  model_year               model  condition  cylinders fuel  odometer transmission    type paint_color  is_4wd date_posted  days_listed
33906  12995      1908.0           gmc yukon       good        8.0  gas  169328.0    automatic     SUV       black     NaN  2018-07-06           34
33907  12995      1908.0   cadillac escalade  excellent        8.0  gas       NaN    automatic     SUV       white     NaN  2018-06-24           25
45694  18000      1929.0          ford f-150       good        8.0  gas       NaN       manual   other      silver     NaN  2018-11-18           59
34713   5000      1936.0          ford f-150  excellent        6.0  gas   30000.0       manual  pickup      purple     NaN  2018-11-22           10
22595  21000      1948.0    chevrolet impala   like new        8.0  gas    4000.0    automatic   sedan         red     NaN  2019-01-18           24
36582  44900      1949.0  chevrolet suburban       good        NaN  gas    1800.0    automatic   wagon      oran

The GMC Yukon and Cadillac Escalade were most likely 2008 models. We can let the Ford F-150s and Chevrolet Suburban slide. The Chevrolet Impala wasn't introduced until 1958, so we could adjust that one as well.

In [595]:
# fix model years for those three vehicles
vehicles.loc[[33906, 33907, 22595],'model_year'] = 2008, 2008, 1958

There are still missing values in `model_year` to contend with, but I'll come back to that. First, let's look at the `model` field. I sorted `value_counts()` for the field instead of using `unique()` so I could get a quick look at all the values in alphabetical order. As long as we're okay with different option packages separate, there aren't a lot of true duplicate entries! The exception is that Ford trucks appear both with and without a hyphen (i.e. "f-150" vs. "f150"). We can fix that easily.

In [596]:
# print(vehicles['model'].value_counts().reset_index().sort_values(by='model').to_string())

def hyphenate_trucks(model):
    '''If input string contains unhyphenated Ford model name, replace with hyphenated version'''
    model = model.replace('f150','f-150')
    model = model.replace('f250','f-250')
    model = model.replace('f350','f-350')
    return model

vehicles['model'] = vehicles['model'].apply(hyphenate_trucks)

Now that we've checked out the `model` field, let's return to the missing values in `model_year`. A vehicle model name limits the possible values of `model_year` to the years in which the model was made. We can get a reasonable `model_year` value for vehicles missing it by taking the median `model_year` for other vehicles of the same model.

In [597]:
# We shouldn't calculate a new median every time we need to fill a missing value.
# To start, filter vehicles to models in rows without model_year, then drop duplicates
# to get a Series with unique model names for which we need to get a median, then DataFrame that.
model_median_year = vehicles.query('model_year.isna()')['model'].drop_duplicates().to_frame()

def get_median_year(m):
    '''Return median model year for a given model in vehicles dataframe.'''
    return vehicles.query('model == @m')['model_year'].median()

# Add a second column to DataFrame by passing model names from the first column to the above function.
model_median_year['median_year'] =  model_median_year['model'].apply(get_median_year)

# Reduce DataFrame to an indexed Series so I can use it for mapping.
# Maybe there's a more graceful way to do this?
model_median_year = model_median_year.set_index('model').median_year

# Got technique for creating map from the answer to this question, but application steamrolls existing values:
# https://stackoverflow.com/questions/54248650/fill-missing-values-based-on-another-column-in-a-pandas-dataframe
# The answer for this question had correct technique to replace only missing values:
# https://stackoverflow.com/questions/44763136/how-to-map-missing-values-of-a-dfs-column-according-to-another-columns-values
vehicles['model_year'] = vehicles['model_year'].fillna(vehicles['model'].map(model_median_year))

The `model_year` column has been filled out; we found a median year for everything. We've already corrected errors in the `model` column. There are no missing values in the `condition` column; we'll have to trust it. Next is `cylinders`. The values look reasonable. Cars of the same model are likely to have the same number of cylinders, so the same median-finding approach should be appropriate here.

In [598]:
model_median_cylinders = vehicles.query('cylinders.isna()')['model'].drop_duplicates().to_frame()
def get_median_cylinders(m):
    '''Return median cylinder value for a given model in vehicles dataframe.'''
    return vehicles.query('model == @m')['cylinders'].median()
model_median_cylinders['median_cylinders'] = model_median_cylinders['model'].apply(get_median_cylinders)
model_median_cylinders = model_median_cylinders.set_index('model').median_cylinders
vehicles['cylinders'] = vehicles['cylinders'].fillna(vehicles['model'].map(model_median_cylinders))

The next column is `fuel`. The value for some vehicles is "other," which isn't very helpful, but otherwise there are no missing values. The next column, `odometer`, is missing some values. To fill missing cells with representative odometer values, we can find another median. The odometer reading is usually higher the longer a vehicle has been on the road, so we'll get median mileage for vehicles' model years instead of getting median mileage for different models.

In [603]:
myear_med_odo = vehicles.query('odometer.isna()')['model_year'].drop_duplicates().to_frame()
def get_median_odo(my):
    '''Return median odometer value for a given model year in vehicles dataframe.'''
    median_odo = vehicles.query('model_year == @my')['odometer'].median()
    # catch the one case where median_odo is NaN because there are no other cars in model year; return 0
    if median_odo == median_odo:
        return median_odo
    else:
        return 0
myear_med_odo['median_odo'] = myear_med_odo['model_year'].apply(get_median_odo)
myear_med_odo = myear_med_odo.set_index('model_year').median_odo
vehicles['odometer'] = vehicles['odometer'].fillna(vehicles['model_year'].map(myear_med_odo))

This takes care of completely missing odometer values. As far as data quality goes, there are again some suspect values -- cars not in "like new" condition with `odometer` values of 0, or cars with `odometer` values above 300,000 that are unlikely to have actually lasted so long. Again, we could attempt to correct for this by cherry-picking, but this information isn't critical to the present analysis anyway, so we may as well leave it; we can't be certain that it's wrong.

`transmission` also has no missing values, although again, the value is 'other' for a handful of vehicles. `type` is also 100% non-null, but let's look at the values for a moment:

In [609]:
display(vehicles['type'].value_counts())

display(vehicles[vehicles['type'] == 'truck']['model'].value_counts().tail(10))

type
SUV            12100
truck          12038
sedan          11874
pickup          6973
coupe           2223
wagon           1525
mini-van        1159
hatchback       1026
van              613
convertible      434
other            255
offroad          212
bus               24
Name: count, dtype: int64

model
chevrolet corvette            1
honda pilot                   1
gmc yukon                     1
toyota corolla                1
jeep wrangler unlimited       1
nissan altima                 1
chevrolet malibu              1
ford escape                   1
jeep grand cherokee laredo    1
toyota camry                  1
Name: count, dtype: int64

The first thing I'd like to point out is that one of the most frequent `type` values, "truck," is ambiguous. Is a pickup truck a "pickup," or is it a "truck?" Do SUVs count as "trucks?" As it turns out, when I list model value counts for the "truck" category, there's no shortage of vehicles that aren't trucks at all: Chevrolet Corvette, Toyota Corolla, Nissan Altima... Vehicle type is important to my analysis, but checking the `type` column for accuracy by checking the type against the model isn't worth the trouble. I'll filter my data to pickup trucks by `model` column values later, instead.

The next column, `paint_color`, has missing values, but it's probably not helpful to guess vehicle color based on the most popular colors, so I'll fill null values with "unknown."

In [610]:
vehicles['paint_color'].fillna('unknown',inplace=True)

Next is `is_4wd`. Unfortunately, it appears this column either contains '1' or a missing value; there's no way to distinguish between vehicles that are not 4WD and vehicles that are missing data. I'll fill missing values with zeroes; this is probably a safer bet than trying to predict if a vehicle is 4WD based on other vehicles of the same model being described as 4WD.

In [611]:
vehicles['is_4wd'].fillna(0,inplace=True)

The `date_posted` and `days_listed` columns are also complete. I'll finish by changing the float-type columns to int-type, since they all represent discrete values.

In [613]:
vehicles['model_year'] = vehicles['model_year'].astype(int)
vehicles['cylinders'] = vehicles['cylinders'].astype(int)
vehicles['odometer'] = vehicles['odometer'].astype(int)
vehicles['is_4wd'] = vehicles['is_4wd'].astype(int)

vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50456 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         50456 non-null  int64         
 1   model_year    50456 non-null  int64         
 2   model         50456 non-null  object        
 3   condition     50456 non-null  object        
 4   cylinders     50456 non-null  int64         
 5   fuel          50456 non-null  object        
 6   odometer      50456 non-null  int64         
 7   transmission  50456 non-null  object        
 8   type          50456 non-null  object        
 9   paint_color   50456 non-null  object        
 10  is_4wd        50456 non-null  int64         
 11  date_posted   50456 non-null  datetime64[ns]
 12  days_listed   50456 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(6)
memory usage: 6.4+ MB


#### Duplicate values in the data

There are no *exact* duplicate lines. However, a single vehicle may appear in the dataset more than once:

In [614]:
vehicles.query('odometer == 151248').sort_values('date_posted')

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
27375,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,0,2018-09-25,72
34389,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,0,2019-02-02,28
33434,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,0,2019-02-05,102
1309,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,black,0,2019-03-02,56
1668,189000,2014,ford f-150,good,6,gas,151248,automatic,truck,unknown,0,2019-03-20,21


We don't have the benefit of unique identifiers like VINs in our data, but it seems likely that where the year, model, and exact odometer reading on several lines are identical, those lines are referring to the same vehicle. The lines above are mostly identical, except that the `paint_color` was missing from one line, and the `date_posted` and `days_listed` values vary. The posting dates and number of days listed for this particular vehicle would have overlapped. Perhaps advertisements in this dataset were pulled from more than one source.

As such, `days_listed` doesn't tell us anything about how long a car has been for sale. The latest advertisement for this truck may have been up for 21 days, but it could already be 6 months from the date the first advertisement for it went up. Given a set of duplicate rows, a more descriptive `days_listed` could be calculated by finding the number of days between the oldest and newest recorded ads, then adding the `days_listed` value corresponding to the newest ad. That's beyond the scope of this project, though, so instead I'll sort the DataFrame by most recent `date_posted` and drop duplicate rows associated with older advertisements.

In [615]:
vehicles.sort_values('date_posted',inplace=True)

vehicles.drop_duplicates(['model_year','model','odometer'], keep='last', inplace=True)

### Selecting data for analysis

For this visualization, I'd like to look specifically at pickup trucks. I noticed earlier that the data in the `type` column is not accurate enough to filter by, so instead I'll make my own list of pickup models to include in my analysis. First, what makes are represented in the dataset?

In [616]:
def getmake(makemodel):
    '''Get the first word from a string combining make and model.'''
    make = makemodel.split(' ',1)[0]
    return make

# Go through the data getting makes. Return unique results.
print(vehicles['model'].apply(getmake).unique())

['honda' 'nissan' 'chevrolet' 'ram' 'jeep' 'hyundai' 'gmc' 'ford' 'toyota'
 'dodge' 'volkswagen' 'buick' 'subaru' 'bmw' 'kia' 'chrysler' 'acura'
 'cadillac' 'mercedes-benz']


Most of these automakers have made a pickup truck at least once, but it's likely that only a few of them will be relevant to an analysis of pickup trucks in our dataset. Filtering `model` by make, I used `value_counts()` to look through short lists of models by maker.

There are no pickups in our dataset made by BMW, Hyundai, Chrysler, Honda, Jeep, Subaru, Mercedes-Benz, Acura, Cadillac, Volkswagen, or Buick.

I am concerned with the following models:
- Ford: F-150, F-250, F-350, Ranger
- Toyota: Tacoma, Tundra
- Chevrolet: Silverado, Colorado
- Ram: 1500, 2500, 3500
- GMC: Sierra
- Nissan: Frontier
- Dodge: Dakota

We corrected earlier for punctuation differences among Ford truck models. The format of the `model` field is consistent enough that I should now be able to limit the dataset to pickups by checking the `model` field for lowercase make/model combinations that correspond to pickup models. I'll filter these rows into a new `pickups` dataframe, and give `make` its own column for graphing purposes.

In [617]:
pickup_models = ['ford f-150', 'ford f-250', 'ford f-350', 'ford ranger', 'toyota tacoma', 'toyota tundra', \
                 'chevrolet colorado', 'chevrolet silverado', 'ram 1500', 'ram 2500', 'ram 3500', 'gmc sierra', \
                 'nissan frontier', 'dodge dakota']

# https://stackoverflow.com/questions/61158898/filter-pandas-where-some-columns-contain-any-of-the-words-in-a-list
# Select only rows where 'model' field contains a string from the list of known pickup_models.
pickups = vehicles[vehicles['model'].str.contains('|'.join(pickup_models))]

pickups['make'] = pickups['model'].apply(getmake)

/tmp/ipykernel_299805/2025677828.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Let's plot a couple of histograms to show how the maker of a truck affects the distribution of advertised prices.

In [618]:
big3price = pickups.query("make in ['ford', 'chevrolet', 'ram']")[['price','make']]
big3price_hist = px.histogram(big3price, x="price", color="make", barmode="overlay",title="Price Frequencies by Make, Three Major Automakers")
big3price_hist.update_layout(yaxis_title_text = 'Number of Trucks Advertised',xaxis_title_text = 'Price')

The median price for Chevrolet trucks appears to be a little higher than it is for Ford. There are fewer Ram truck advertisements in our dataset, but the median sale price for these is even higher. There are proportionately more lower-end trucks advertised for Ford and Chevrolet than there are for Ram.

Let's take the Ram price histogram and compare it with Dodge and Nissan. The histograms for these makes will look substantially different from the other ones.

In [620]:
big3price = pickups.query("make in ['ram','dodge','nissan'] & price < 150000")[['price','make']]
big3price_hist = px.histogram(big3price, x="price", color="make", barmode="overlay",title="Price Frequencies by Make: Ram, Dodge, Nissan")
big3price_hist.update_layout(yaxis_title_text = 'Number of Trucks Advertised',xaxis_title_text = 'Price')

Dodge is firmly at the low end, but that's because the only model classified as Dodge in our dataset is the Dodge Dakota, which is an older model. Ram is a offshoot of the Dodge brand; if we treat Ram and Dodge as being the same maker, the older Dodge trucks would fill out the lower end of the Ram histogram and make the distribution look a little more like Chevrolet or Ford.

Nissan, on the other hand, is interesting -- sale prices never seem to be above \$25000, half the asking price for some Ram trucks, but the distribution doesn't skew towards the lower end. Could it be that Nissan trucks retain value, even if the base value is lower overall? Let's look at a scatter plot.

In [621]:
scatterfig = px.scatter(pickups.query('price < 100000 & model_year >= 1980'), x='model_year', y='price', color='make', hover_data=['model','condition','odometer'], title='Advertised Sale Price by Model Year, All Pickup Makes')
scatterfig.update_layout(yaxis_title_text='Price',xaxis_title_text='Model Year')
scatterfig.show()

(Vehicles made before 1980, and those with sale prices above \$100000, have been excluded from this chart to make it more readable.)

We expect to see lower sale prices for older cars, but if Nissan trucks "retained value" especially well, on a graph like this, the overall slope of the scatter cloud would be less inclined than that of competing makes. That doesn't seem to be the case overall; however, Nissan value does seem to hold its own compared to Dodge trucks for the same model year. (Ram trucks are more expensive to begin with.) Actually, it looks like Chevrolet has some of the highest sale prices for trucks made before 1995 -- they may be the value retention champion.

### Conclusions

The first charts we looked at explored the relationship between pickup truck make and advertised sale price. There is some variation in distribution between makes; more Ford trucks and fewer Dodge trucks appear at the low end of the price distribution. After looking at data for some less popular makes, though, another question arose: what make of trucks retain value? The scatter plot displays the relationship between truck model year and advertised sale price, and by looking at different makes overlaid upon one another, I found that Chevrolet in particular seems to sell for relatively higher prices among pre-1995 trucks. 